In [29]:
import requests
import json

city = "广州市"
# Overpass API端点
url = "https://overpass-api.de/api/interpreter"

query = f"""
[out:json];
area[name="{city}"]->.searchArea;
(
  relation(area.searchArea)["boundary"="administrative"]["admin_level"="6"];
);
out tags;
"""

# 发送请求
response = requests.get(url, params={'data': query})
data = response.json()

# 提取区名
districts = set()
for element in data['elements']:
    if 'tags' in element and 'name' in element['tags']:
        districts.add((element['tags']['name'], element['id']))

print(f"{city}下级行政区有：")
for district in sorted(districts):
    print(district)

广州市下级行政区有：
('从化区', 3287298)
('南沙区', 3287345)
('增城区', 3287299)
('天河区', 3987435)
('海珠区', 3287615)
('番禺区', 3287618)
('白云区', 3987393)
('花都区', 3287616)
('荔湾区', 3287617)
('越秀区', 3987394)
('黄埔区', 3987437)


In [26]:
test_id = 3287618 + 3600000000
test_query = f"""
[out:json][timeout:60];
area({test_id})->.searchArea;
(
  relation(area.searchArea)["boundary"="administrative"]["admin_level"~"7|8"];
);
out tags;
"""

response = requests.post(url, data={"data": test_query})
data = response.json()

# 提取下级行政单位（街道/社区）名称
districts = set()
for element in data['elements']:
    if 'tags' in element and 'name' in element['tags']:
        name = element['tags']['name']
        level = element['tags'].get('admin_level', 'unknown')
        districts.add((name, level, element['id']))

# 输出
print("📍 番禺区下辖街道或社区：")
for district in sorted(districts):
    print(f"✅ {district[0]}（admin_level={district[1]}，relation_id={district[2]}）")


📍 番禺区下辖街道或社区：
✅ 东环街道（admin_level=8，relation_id=12033869）
✅ 化龙镇（admin_level=8，relation_id=12043253）
✅ 南村镇（admin_level=8，relation_id=12034602）
✅ 大石街道（admin_level=8，relation_id=12042955）
✅ 大龙街道（admin_level=8，relation_id=12033891）
✅ 小谷围街道（admin_level=8，relation_id=9463947）
✅ 市桥街道（admin_level=8，relation_id=12031258）
✅ 新造镇（admin_level=8，relation_id=12043044）
✅ 桥南街道（admin_level=8，relation_id=11996533）
✅ 沙头街道（admin_level=8，relation_id=12032052）
✅ 沙湾街道（admin_level=8，relation_id=12032277）
✅ 洛浦街道（admin_level=8，relation_id=9463946）
✅ 石壁街道（admin_level=8，relation_id=12045940）
✅ 石楼镇（admin_level=8，relation_id=12045937）
✅ 石碁镇（admin_level=8，relation_id=12037949）
✅ 钟村街道（admin_level=8，relation_id=12034493）


In [27]:
# 通过街道搜路
xiaoguwei_id = 9463947 + 3600000000
query = f"""
[out:json][timeout:90];
area({xiaoguwei_id})->.searchArea;

(
  way["highway"]["name"~"路$|大道$|巷$|街$|道$|里$|弄$|胡同$|径$|东$|西$|北$|南$"](area.searchArea);
);
out tags;
"""
response = requests.post(url, data={"data": query})

data = response.json()
road_names = []
for element in data.get('elements', []):
    name = element.get('tags', {}).get('name')
    if name and name not in road_names:
        road_names.append(name)
    if len(road_names) >= 10:
        break

print("📍 小谷围街道的前 10 条道路名称：")
for i, road in enumerate(road_names, 1):
    print(f"{i}. {road}")

📍 小谷围街道的前 10 条道路名称：
1. 大学城中环西路
2. 大学城华师一路
3. 环校西路
4. 大学城广大路
5. 档案馆路
6. 大学城外环西路
7. 大学城内环西路
8. 大学城环湖路
9. 科普路
10. 环校北路


In [30]:
# 如果街区没有id

# 直接对区域进行查询
haifx_id = 3282607 + 3600000000

query = f"""
[out:json][timeout:90];
area({haifx_id})->.searchArea;

(
  way["highway"]["name"~"路$|大道$|巷$|街$|道$|里$|弄$|胡同$|径$|东$|西$|北$|南$"](area.searchArea);
);
out tags;
"""
response = requests.post(url, data={"data": query})

data = response.json()
road_names = []
for element in data.get('elements', []):
    name = element.get('tags', {}).get('name')
    if name and name not in road_names:
        road_names.append(name)
    if len(road_names) >= 10:
        break

print("📍 海丰县的前 10 条道路名称：")
for i, road in enumerate(road_names, 1):
    print(f"{i}. {road}")

📍 海丰县的前 10 条道路名称：
1. 海银路
2. 海银公路
3. 128县道
4. 汕尾大道北
5. 二环路
6. 迎新路
7. 解放北路
8. 海龙路
9. 海紫路
10. 向阳路
